# Trial sos4py for the Australian BoM Water Data Online service

Adapting a demo notebook found in the package [sos4py](https://github.com/52North/sos4py)


In [1]:
from sos4py.main import connection_sos
from sos4py.sos_2_0_0 import SOSGetFeatureOfInterestResponse

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd

import folium # avail on conda-forge
# note, had an issue with 
# rasterio leading to ImportError: libcrypto.so.1.0.0: cannot open shared object file: No such file or directory
# needed to `mamba update -c conda-forge rasterio` to get it back into shape. May have been because I had used the pytorch repo as well. May have.
import contextily as ctx

In [2]:
# Create SOS instance 
# fluggs_sos = connection_sos("https://fluggs.wupperverband.de/sos2/service")
fluggs_sos = connection_sos("http://www.bom.gov.au/waterdata/services?service=SOS&version=2.0&request=GetDataAvailability")

# 1) Learn about content of SOS 

#### Service

In [3]:
print(fluggs_sos.sosServiceIdentification())

_root                 [[], [], [], [], [], [], [], [], [], [], [], []]
title                                               KISTERS KiWIS SOS2
abstract                                                          None
keywords                                                            []
accessconstraints                    maxNumberOfReturnedValues=2000000
fees                                                              None
type                                                               SOS
service                                                            SOS
version                                                          2.0.0
versions                                                       [2.0.0]
profiles             [http://www.opengis.net/spec/SOS/2.0/conf/core...
Name: ServiceIdentification, dtype: object


In [4]:
print(fluggs_sos.sosServiceIdentification()['profiles'])

['http://www.opengis.net/spec/SOS/2.0/conf/core', 'http://www.opengis.net/spec/SOS/2.0/conf/soap', 'http://www.opengis.net/spec/SOS/2.0/conf/kvp-core', 'http://www.opengis.net/spec/SOS/2.0/conf/foiRetrieval', 'http://www.opengis.net/spec/SOS/2.0/conf/gda', 'http://www.opengis.net/spec/SOS/2.0/conf/xml', 'http://www.opengis.net/spec/SOS_application-profile_hydrology/1.0/req/hydrosos']


#### Provider

In [5]:
print(fluggs_sos.sosProvider())

_root           [[[<Element '{http://www.opengis.net/ows/1.1}A...
name                                                         None
organization                                                 None
site                                                         None
role                                                         None
position                                                     None
phone                                                        None
fax                                                          None
address                                                      None
city                                                         None
region                                                       None
postcode                                                     None
country                                                      None
email                                                        None
url                                                          None
hours     

#### SOS operations

In [6]:
print(fluggs_sos.sosOperationsMetadata())
#print(fluggs_sos.sosOperationsMetadata()[1]['Name'])

[Name                                                DescribeSensor
FormatOptions                                           [text/xml]
Parameters                                                      {}
Methods          [{'constraints': [Constraint: Content-Type - [...
Constraints                                                     []
Name: DescribeSensor, dtype: object, Name                                               GetCapabilities
FormatOptions                                           [text/xml]
Parameters                                                      {}
Methods          [{'constraints': [Constraint: Content-Type - [...
Constraints                                                     []
Name: GetCapabilities, dtype: object, Name                                           GetDataAvailability
FormatOptions                                           [text/xml]
Parameters                                                      {}
Methods          [{'constraints': [Constraint: Conten

#### All offerings

In [7]:
sos_off = fluggs_sos.sosOfferings()

In [8]:
type(sos_off), len(sos_off)

(list, 193)

In [9]:
sos_off[1]

_root                            [[], [], [], [], [[<Element '{http://www.openg...
id                               http://bom.gov.au/waterdata/services/tstypes/P...
description                      Offering for timeseries type Harmonised.Combin...
name                                                Harmonised.Combined.AsStored.1
bbox                                  (-159.6879, -90.0, 168.80790000000002, 90.0)
bbox_srs                                                urn:ogc:def:crs:EPSG::4326
begin_position                                                                None
end_position                                                                  None
procedures                       [http://bom.gov.au/waterdata/services/tstypes/...
procedure_description_formats                                                   []
observed_properties                                                             []
features_of_interest                                                            []
resp

#### All available phenomena (independent of e.g feature of interest or offering)

In [10]:
sos_ph = fluggs_sos.sosPhenomena()

sos_ph

[]

list of phenomena not found... Reverse engineering sos4py method `sosPhenomena` : 

In [11]:
from sos4py.sos_2_0_0 import namespaces

from owslib.util import testXMLValue, testXMLAttribute, nspath_eval, openURL
from owslib.swe.observation.sos200 import SensorObservationService_2_0_0
from owslib.swe.observation.sos200 import SOSGetObservationResponse
from owslib.swe.observation.waterml2 import MeasurementTimeseriesObservation
from owslib.swe.observation.om import MeasurementObservation
from owslib.etree import etree
from owslib import ows
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pyproj
import inspect
# from .util import get_namespaces, nspv, TimePeriod, parseGDAReferencedElement, gda_member, check_list_param

In [12]:
_capabilities= fluggs_sos._capabilities

In [13]:
_capabilities

<Element '{http://www.opengis.net/sos/2.0}Capabilities' at 0x7fbef631d360>

In [15]:
_capabilities.findall(nspath_eval('sos:contents/sos:Contents/swes:offering/sos:ObservationOffering/swes:observableProperty', namespaces))

[]

Using the VSCode extension "XML Tools" to find the "swes:observableProperty" thinggy which appears to be under another path in the Kisters' server. 

In [19]:
x = _capabilities.findall(nspath_eval('sos:contents/sos:Contents/swes:observableProperty', namespaces))

In [20]:
[y.text for y in x]

['http://bom.gov.au/waterdata/services/parameters/Water Course Discharge',
 'http://bom.gov.au/waterdata/services/parameters/Water Course Level',
 'http://bom.gov.au/waterdata/services/parameters/Rainfall',
 'http://bom.gov.au/waterdata/services/parameters/Ground Water Level',
 'http://bom.gov.au/waterdata/services/parameters/Dry Air Temperature',
 'http://bom.gov.au/waterdata/services/parameters/Relative Humidity',
 'http://bom.gov.au/waterdata/services/parameters/Wind Speed',
 'http://bom.gov.au/waterdata/services/parameters/Electrical Conductivity @ 25C',
 'http://bom.gov.au/waterdata/services/parameters/Evaporation',
 'http://bom.gov.au/waterdata/services/parameters/Wind Direction',
 'http://bom.gov.au/waterdata/services/parameters/Turbidity',
 'http://bom.gov.au/waterdata/services/parameters/Water Temperature',
 'http://bom.gov.au/waterdata/services/parameters/pH',
 'http://bom.gov.au/waterdata/services/parameters/Storage Volume',
 'http://bom.gov.au/waterdata/services/parameters/

#### All available features of interest

In [23]:
print(fluggs_sos.sosFeaturesOfInterest())

KeyError: 'featureOfInterest'

In [26]:
get_foi = fluggs_sos.get_operation_by_name('GetFeatureOfInterest')
get_foi

<owslib.ows.OperationsMetadata GetFeatureOfInterest at 0x7fbef61f0550>

In [32]:
dir(get_foi)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'constraints',
 'formatOptions',
 'methods',
 'name',
 'parameters']

In [27]:
get_foi.parameters

{}

In [30]:
get_foi.formatOptions

['text/xml']

In [25]:
get_foi.methods

[{'constraints': [Constraint: Content-Type - ['application/x-kvp']],
  'type': 'Get',
  'url': 'http://wdo-prod-01/waterdata/services?datasource=0'},
 {'constraints': [Constraint: Content-Type - ['application/soap+xml', 'application/xml']],
  'type': 'Post',
  'url': 'http://wdo-prod-01/waterdata/services?datasource=0'}]


## Rest below not yet adapted from original example


#### Data availability

In [ ]:
#print(fluggs_sos.get_data_availability())

props = ['Wassertemperatur']
fois = ['Opladen','Bever-Talsperre']
gda = fluggs_sos.get_data_availability(observedProperties=props,featuresOfInterest=fois)
print(gda)

# 2) Spatial data

### 2a) Get spatial data

In [ ]:
include_phenomena = False
fluggs_sites = fluggs_sos.get_sites(include_phenomena)
fluggs_sites.head()

In [ ]:
# Check and change coordinate reference system
print(fluggs_sites.crs,"\n")
fluggs_sites = fluggs_sites.to_crs('epsg:4326')

# View specific site using integer-locaiton
print(fluggs_sites.iloc[0])

### 2b) Plot spatial data

In [ ]:
# Static map using GeoDataframe.plot

# sites = sites.to_crs('epsg:3857') # epsg:4326
ax = fluggs_sites.plot(figsize=(15, 15))
ctx.add_basemap(ax, crs=fluggs_sites.crs.to_string())
# optional parameters:
# source=ctx.providers.OpenStreetMap.Mapnik
# zoom=12
ax

In [ ]:
# Interactive map using folium

lngs = list(fluggs_sites['geometry'].apply(lambda coord: coord.x))
lats = list(fluggs_sites['geometry'].apply(lambda coord: coord.y))
avg_lat = sum(lats) / len(lats)
avg_lngs = sum(lngs) / len(lngs)

# folium uses (lat, long) or (y, x), respectively
locationlist = [[site.y, site.x] for site in fluggs_sites['geometry']]

m = folium.Map(location=[avg_lat, avg_lngs], zoom_start=10)
for point in range(0, len(locationlist)):
    popup = folium.Popup(folium.IFrame(html=fluggs_sites['site_name'][point], width=200, height=80))
    folium.Marker(locationlist[point], popup=popup).add_to(m)
m

# 3) Sensor data

### 3a) Get sensor data

In [ ]:
# Optional arguments for get_data(): procedures, phenomena, sites, begin, end
begin = '2019-01-01T00:00:00Z' 
end = '2019-01-31T23:59:59Z'
procedures = ['2m_Tiefe']
phenomena = ['Wassertemperatur']
sites = ['Opladen']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)

fluggs_obs.head()

In [ ]:
# Check unique values in column
fluggs_obs['procedure'].unique()

In [ ]:
# Subset of data frame
#fluggs_obs[(fluggs_obs['site']=='Opladen') & (fluggs_obs['procedure']=='Einzelwerte')]
#fluggs_obs[(fluggs_obs['value'] > 5)] 

### 3b) Plot sensor data

#### Plot a single time series

In [ ]:
foi = sites[0]
x = fluggs_obs[fluggs_obs['site']==foi]['time_stamp'].to_numpy()
y = fluggs_obs[fluggs_obs['site']==foi]['value'].to_numpy()

plt.figure(figsize=(12,5))
plt.plot(x,y)
plt.xlabel('Time')
plt.ylabel(fluggs_obs['phenomenon'][0] + " " + fluggs_obs['procedure'][0] + " in " + fluggs_obs['unit'][0])
plt.title(fluggs_obs['site'][0])
plt.show()

#### Plot two time series of two different sites

In [ ]:
# Check data availability
fluggs_sos.get_data_availability(observedProperties=['Wassertemperatur'],procedures=['Einzelwerte'])

In [ ]:
# New request with two sites
begin = '2019-01-01T00:00:00Z' 
end = '2019-01-31T23:59:59Z'
procedures = ['Einzelwerte']
phenomena = ['Wassertemperatur']
sites = ['Opladen','Laaken']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)
fluggs_obs.head()

In [ ]:
x1 = fluggs_obs[(fluggs_obs['site']==sites[0])]['time_stamp'].to_numpy()
y1 = fluggs_obs[(fluggs_obs['site']==sites[0])]['value'].to_numpy()

x2 = fluggs_obs[(fluggs_obs['site']==sites[1])]['time_stamp'].to_numpy()
y2 = fluggs_obs[(fluggs_obs['site']==sites[1])]['value'].to_numpy()

plt.figure(figsize=(12,5))
plt.plot(x1,y1,label=sites[0])
plt.plot(x2,y2,label=sites[1])
plt.xlabel('Time')
plt.ylabel(fluggs_obs['phenomenon'][0] + " " + fluggs_obs['procedure'][0] + " in " + fluggs_obs['unit'][0])
plt.legend()
plt.show()

#### Plot two time series of two different phenomena

In [ ]:
fluggs_sos.get_data_availability(featureOfInterest=['Bever-Talsperre'],observedProperties=['Luftfeuchte','Lufttemperatur'])

In [ ]:
# New request with two phenomena for one site
begin = '2011-01-01T00:00:00Z' 
end = '2011-01-31T23:59:59Z'
phenomena = ['Luftfeuchte','Lufttemperatur']
sites = ['Bever-Talsperre']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)
fluggs_obs.head()

In [ ]:
x3 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]['time_stamp'].to_numpy()
y3 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]['value'].to_numpy()

x4 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]['time_stamp'].to_numpy()
y4 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]['value'].to_numpy()

fig, ax1 = plt.subplots(figsize=(12,5))

color = 'red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Luftfeuchte', color=color)
ax1.plot(x3, y3, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'blue'
ax2.set_ylabel('Lufttemperatur', color=color)  
ax2.plot(x4, y4, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

#### Data sub-sampling

In [ ]:
interval = '4h'

# Phenomenon 1
df1 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]
df1 = df1[['time_stamp','value']]

# Remove duplicate time stamps and sub-sample data
grouped = df1.groupby('time_stamp')['value']
grouped = grouped.agg('mean')
df1 = pd.DataFrame(grouped)
df1_sub = df1.resample(interval).nearest()
df1_sub = df1_sub.reset_index()


# Phenomenon 2
df2 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]
df2 = df2[['time_stamp','value']]

# Remove duplicate time stamps and sub-sample data
grouped = df2.groupby('time_stamp')['value']
grouped = grouped.agg('mean')
df2 = pd.DataFrame(grouped)
df2_sub = df2.resample(interval).nearest()
df2_sub = df2_sub.reset_index()

In [ ]:
x3 = df1_sub['time_stamp'].to_numpy()
y3 = df1_sub['value'].to_numpy()

x4 = df2_sub['time_stamp'].to_numpy()
y4 = df2_sub['value'].to_numpy()

fig, ax1 = plt.subplots(figsize=(12,5))

color = 'red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Luftfeuchte', color=color)
ax1.plot(x3, y3, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'blue'
ax2.set_ylabel('Lufttemperatur', color=color)  
ax2.plot(x4, y4, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

#### Histogram and Kernel density estimation (KDE)

In [ ]:
sns.distplot(y1, bins=10);
sns.distplot(y2, bins=10);

#### 2D KDE

In [ ]:
sns.kdeplot(y1,y2);

#### Joint plots/Correlation

In [ ]:
sns.jointplot(y1, y2, kind='reg', joint_kws={'line_kws':{'color':'red'}}).annotate(stats.pearsonr);

In [ ]:
sns.jointplot(y1, y2, kind='hex');

In [ ]:
sns.jointplot(y1, y2, kind='kde');